#### In this Notebook we will be calculating the weights for the stocks in the portfolio using the new Filter methods. 

In [11]:
import ffn
import pandas as pd
import numpy as np
from numpy import linalg
from scipy.optimize import minimize
import cvxpy as cp
import copy
cp.settings.EIGVAL_TOL = 1e-08

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
data = pd.read_csv("LogReturn_v2.csv", parse_dates=['Date'], index_col = 'Date')
data

,AAPL,ABBV,ADBE,AMZN,BKNG,BUD,BX,C,CHH,CICHY,...,TCOM,TMO,TMUS,TRIP,TXN,UL,UNH,V,VRTX,WFC
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-05,-0.028576,-0.018998,-0.004989,-0.020731,-0.039726,-0.033211,-0.014758,-0.032022,-0.019475,0.004226,...,-0.014254,-0.013832,-0.013646,-0.025252,-0.015641,-0.018938,-0.016609,-0.022321,-0.016715,-0.027805
2015-01-06,0.000094,-0.004962,-0.020350,-0.023098,-0.016184,-0.002223,-0.019517,-0.035839,-0.016679,-0.010294,...,0.012926,-0.009382,-0.002230,-0.038116,-0.016663,-0.005111,-0.002020,-0.006465,-0.030792,-0.021085
2015-01-07,0.013925,0.039621,0.008190,0.010544,-0.009667,0.014452,0.014151,0.009228,0.022361,0.021079,...,0.015600,0.029517,0.050083,0.011786,0.017801,0.010703,0.010158,0.013309,0.027731,0.005934
2015-01-08,0.037703,0.010405,0.025135,0.006813,0.012340,0.031132,0.007743,0.014936,0.032011,-0.005977,...,0.014501,0.010339,0.026892,0.033591,0.016184,0.018336,0.046629,0.013324,0.027063,0.021896
2015-01-09,0.001072,-0.027736,-0.014922,-0.011818,-0.028941,-0.008094,0.014723,-0.022587,0.002651,-0.007220,...,0.005785,-0.004000,0.007894,0.017187,0.000560,-0.008247,-0.009404,-0.014934,-0.014299,-0.016567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-23,-0.015239,0.000350,-0.008747,-0.030553,-0.029910,-0.040548,-0.022499,-0.029388,-0.001787,0.000845,...,0.015119,0.000961,-0.020346,-0.017970,-0.008212,-0.040448,-0.007468,-0.009899,-0.010144,-0.027098
2022-09-26,0.002258,-0.013016,-0.027071,0.011969,-0.000377,-0.023867,-0.004080,-0.029114,-0.019008,-0.009334,...,0.051736,-0.012521,-0.000076,-0.010023,-0.005159,-0.000684,-0.010274,-0.018489,-0.023813,-0.009948
2022-09-27,0.006545,0.003605,0.002200,-0.006447,-0.000054,-0.008379,-0.014657,-0.009348,0.013533,-0.020673,...,0.010540,-0.011046,0.002717,0.020395,0.001557,-0.011707,0.000020,-0.015176,0.028353,0.000500


In [9]:
data.isnull().values.any()

False

In [10]:
def MVopt(df, risk = 1):
    x0 = np.ones(df.shape[1])/np.sum(np.ones(df.shape[1]))
    r = 63 * np.array(np.mean(df, axis=0)) # Quarterly Return 
    cov = 63 * np.cov(df.transpose()) # Quarterly Covariance 
    
    # Mean-Variance Model
    def MV(x): # x is an array
        return - ( r.T @ x - risk * (x @ cov @ x.T  ) )
    
    # Find the minimum solution of Mean-Variance Model
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},
                   {'type': 'ineq', 'fun': lambda x: x})
    res = minimize(MV, 
                   x0=x0, 
                   method='SLSQP',
                   constraints=constraints, 
                   tol=1e-9, 
                   options={'maxiter': 1500, 'ftol': 1e-9, 'disp': True})
    # OptimizeResult: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.OptimizeResult.html#scipy.optimize.OptimizeResult
    return res.x

In [12]:
def MVCvxOpt(df, x0, risk = 1):
    x0 = np.ones(df.shape[1])/np.sum(np.ones(df.shape[1]))
    r = 63 * np.array(np.mean(df, axis=0)) # Quarterly Return 
    cov = 63 * np.cov(df.transpose()) # Quarterly Covariance
    
    # solve optimization problem by CVXPY
    x = cp.Variable(len(x0))
    constraints = [sum(x) == 1, x >= 0] 
    function = r.T @ x - risk * (cp.quad_form(x, cov)) 
    obj = cp.Maximize(function) # form objective
    prob = cp.Problem(obj, constraints) # form problem
    try:
        res = prob.solve(solver=cp.ECOS, max_iters=100000, verbose = True)
    except:
        return MVopt(df, x0, risk, trade)
    
    return x.value # get solution value

In [13]:
def ThresholdFilter(df, mu_min=0.01, weight_max=0.05, sum_up_to_1 = False):
    r = np.prod(df+1, axis=0)**(365/df.index.shape[0]) - 1 # Annualized Return
    w = r.apply(lambda x : weight_max if (x >= mu_min) else 0)
    if sum_up_to_1 == True:
        w = w / np.sum(w)
    return w

In [14]:
def AdaptiveFilter(df, mu_min=0.01, weight_max=0.05, sum_up_to_1 = False):
    r = np.prod(df+1, axis=0)**(365/df.index.shape[0]) - 1 # Annualized Return
    slope = weight_max/ (np.max(r)-mu_min)
    w = r.apply(lambda x : slope*(x-mu_min) if (x >= mu_min) else 0)
    if sum_up_to_1 == True:
        w = w / np.sum(w)
    return w

In [15]:
def Optimization(df, OptModel, risk=1, **kwargs):
    df['Date'] = df.index
    g = df.groupby(pd.Grouper(key='Date', freq='Q'))  # frequency = quarterly
    dfList = [group for _,group in g]
   
    if OptModel== MVopt or OptModel== MVCvxOpt:
        w = pd.DataFrame(columns=['risk', 'End of Period']+ df.columns[:-1].tolist())
        for i in range(len(dfList)):
            dfTemp = pd.DataFrame(dfList[i])
            dfTemp.drop(['Date'], axis=1, inplace = True)
            res = OptModel(dfTemp, risk) 
            w = w.append({'End of Period': dfTemp.index[-1]}, ignore_index=True)
            w.iloc[i,2:] = pd.Series(res)
        w.iloc[:,0] = risk
    elif OptModel== ThresholdFilter or OptModel== AdaptiveFilter:
        if ('mu_min' in kwargs) == False:
            kwargs['mu_min'] = 0.01
        if ('weight_max' in kwargs) == False:
            kwargs['weight_max'] = 0.05
        if ('sum_up_to_1' in kwargs) == False:
            kwargs['sum_up_to_1'] = False
            
        w = pd.DataFrame(columns=['End of Period']+ df.columns[:-1].tolist())
        for i in range(len(dfList)):
            dfTemp = pd.DataFrame(dfList[i])
            dfTemp.drop(['Date'], axis=1, inplace = True)
            res = OptModel(dfTemp, kwargs['mu_min'], kwargs['weight_max'], kwargs['sum_up_to_1'])
            w = w.append({'End of Period': dfTemp.index[-1]}, ignore_index=True)
            w.iloc[i,1:] = pd.Series(res)
    return w

In [16]:
# for Filter Method
df = data
OptModel = AdaptiveFilter # assign a optimization model

weights = pd.DataFrame(columns=['End of Period']+ df.columns[:-1].tolist())  
w = Optimization(df, OptModel, mu_min=0.01, weight_max=0.05, sum_up_to_1=True)
weights = pd.concat([weights, w], ignore_index=True)
weights = pd.DataFrame(weights).set_index('End of Period')
weights

,AAPL,ABBV,ADBE,AMZN,BKNG,BUD,BX,C,CHH,CICHY,...,TCOM,TMO,TMUS,TRIP,TXN,UL,UNH,V,VRTX,WFC
End of Period,,,,,,,,,,,,,,,,,,,,,
2015-03-31,0.050559,0.0,0.004197,0.084476,0.002142,0.028617,0.065585,0.0,0.060741,0.0,...,0.137193,0.013228,0.062623,0.027519,0.022555,0.013988,0.071035,0.0,0.0,0.0
2015-06-30,0.002265,0.073624,0.037938,0.07413,0.0,0.000175,0.0278,0.02768,0.0,0.066791,...,0.116294,0.0,0.122036,0.011492,0.0,0.011663,0.010383,0.008082,0.012782,0.013908
2015-09-30,0.0,0.0,0.004183,0.262218,0.080778,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.017759,0.0,0.0,0.0,0.0,0.033651,0.0,0.0
2015-12-31,0.0,0.013732,0.028284,0.099313,0.002976,0.045685,0.0,0.005863,0.009531,0.002143,...,0.161107,0.034576,0.0,0.099831,0.019848,0.010776,0.001181,0.021798,0.043817,0.010778
2016-03-31,0.011442,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026068,0.0,...,0.0,0.0,0.0,0.0,0.019508,0.020879,0.043844,0.0,0.0,0.0
2016-06-30,0.0,0.028215,0.003507,0.081703,0.0,0.020503,0.0,0.0,0.0,0.033485,...,0.0,0.011243,0.042925,0.0,0.030436,0.018526,0.032387,0.0,0.016584,0.0
2016-09-30,0.049607,0.004417,0.030533,0.04386,0.046785,0.0,0.009136,0.025987,0.0,0.032072,...,0.029115,0.015174,0.015379,0.0,0.028264,0.0,0.0,0.026536,0.0,0.0
2016-12-30,0.004491,0.0,0.0,0.0,0.0,0.0,0.012782,0.079091,0.072979,0.000466,...,0.0,0.0,0.062347,0.0,0.007742,0.0,0.032818,0.0,0.0,0.073734
2017-03-31,0.056567,0.006881,0.063888,0.035735,0.045634,0.005228,0.01813,0.000395,0.019535,0.008295,...,0.049878,0.013476,0.019402,0.0,0.018071,0.043485,0.003397,0.025047,0.174774,0.00139


In [17]:
np.sum(weights, axis=1)

End of Period
2015-03-31    1.0
2015-06-30    1.0
2015-09-30    1.0
2015-12-31    1.0
2016-03-31    1.0
2016-06-30    1.0
2016-09-30    1.0
2016-12-30    1.0
2017-03-31    1.0
2017-06-30    1.0
2017-09-29    1.0
2017-12-29    1.0
2018-03-29    1.0
2018-06-29    1.0
2018-09-28    1.0
2018-12-31    1.0
2019-03-29    1.0
2019-06-28    1.0
2019-09-30    1.0
2019-12-31    1.0
2020-03-31    1.0
2020-06-30    1.0
2020-09-30    1.0
2020-12-31    1.0
2021-03-31    1.0
2021-06-30    1.0
2021-09-30    1.0
2021-12-31    1.0
2022-03-31    1.0
2022-06-30    1.0
2022-09-29    1.0
dtype: float64

In [18]:
weights.to_csv(f'weights_{OptModel.__name__}.csv')

In [19]:
# for Mean-Variance Model
df = data
OptModel = MVCvxOpt # assign a optimization model

weights = pd.DataFrame(columns=['End of Period']+ df.columns.tolist())  
w = Optimization(df, OptModel)
weights = pd.concat([weights, w], ignore_index=True)
weights = pd.DataFrame(weights)

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jul 18 03:42:04 PM: Your problem has 47 variables, 2 constraints, and 0 parameters.
(CVXPY) Jul 18 03:42:04 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 18 03:42:04 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 18 03:42:04 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 18 03:42:04 PM: Compiling problem (target solver=ECOS).
(CVXPY) Jul 18 03:42:04 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> Cone

(CVXPY) Jul 18 03:42:04 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> ECOS
(CVXPY) Jul 18 03:42:04 PM: Applying reduction FlipObjective
(CVXPY) Jul 18 03:42:04 PM: Applying reduction Dcp2Cone
(CVXPY) Jul 18 03:42:04 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 18 03:42:04 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Jul 18 03:42:04 PM: Applying reduction ECOS
(CVXPY) Jul 18 03:42:04 PM: Finished problem compilation (took 6.951e-02 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Jul 18 03:42:05 PM: Invoking solver ECOS  to obtain a solution.
-------------------------------------------------------------------------------
                                    Summary                                    
-----------------------------

(CVXPY) Jul 18 03:42:05 PM: Optimal value: 3.548e-01
(CVXPY) Jul 18 03:42:05 PM: Compilation took 6.660e-02 seconds
(CVXPY) Jul 18 03:42:05 PM: Solver (including time spent in interface) took 4.173e-03 seconds
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jul 18 03:42:05 PM: Your problem has 47 variables, 2 constraints, and 0 parameters.
(CVXPY) Jul 18 03:42:05 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 18 03:42:05 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 18 03:42:05 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
----------------------------------

(CVXPY) Jul 18 03:42:05 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> ECOS
(CVXPY) Jul 18 03:42:05 PM: Applying reduction FlipObjective
(CVXPY) Jul 18 03:42:05 PM: Applying reduction Dcp2Cone
(CVXPY) Jul 18 03:42:05 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 18 03:42:05 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Jul 18 03:42:05 PM: Applying reduction ECOS
(CVXPY) Jul 18 03:42:05 PM: Finished problem compilation (took 4.242e-02 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Jul 18 03:42:05 PM: Invoking solver ECOS  to obtain a solution.
-------------------------------------------------------------------------------
                                    Summary                                    
-----------------------------

 8  -2.997e-01  -2.997e-01  +2e-04  5e-07  3e-07  3e-07  4e-06  0.9538  1e-03   2  1  1 |  0  0
 9  -2.997e-01  -2.997e-01  +1e-05  4e-08  2e-08  1e-08  3e-07  0.9890  6e-02   2  1  1 |  0  0
10  -2.997e-01  -2.997e-01  +3e-07  9e-10  6e-10  3e-10  6e-09  0.9752  1e-04   1  1  1 |  0  0
11  -2.997e-01  -2.997e-01  +2e-08  6e-11  4e-11  2e-11  4e-10  0.9361  4e-03   1  1  1 |  0  0
12  -2.997e-01  -2.997e-01  +3e-10  8e-13  5e-13  3e-13  6e-12  0.9890  2e-03   1  1  1 |  0  0

OPTIMAL (within feastol=7.7e-13, reltol=8.9e-10, abstol=2.7e-10).
Runtime: 0.005609 seconds.


ECOS 2.0.10 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +5.729e-03  -1.976e-01  +7e+01  8e-01  8e-01  1e+00  1e+00    ---    ---    1  1  - |  -  - 
 1  -4.660e-01  -5.418e-01  +2e+01  3e-01  1e-01  1e-01  4e-01  0.7563  5e-02   1  1  1 |  0  0
 2  -2.388e-01  -2.356e-01  +1e+01  9e-02  3e-

...

-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Jul 18 03:42:06 PM: Problem status: optimal
(CVXPY) Jul 18 03:42:06 PM: Optimal value: 1.866e-01
(CVXPY) Jul 18 03:42:06 PM: Compilation took 4.093e-02 seconds
(CVXPY) Jul 18 03:42:06 PM: Solver (including time spent in interface) took 4.579e-03 seconds
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jul 18 03:42:06 PM: Your problem has 47 variables, 2 constraints, and 0 parameters.
(CVXPY) Jul 18 03:42:06 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 18 03:42:06 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 18 03:42:06 PM: CVX

 2  1  1 |  0  0
11  -2.396e-01  -2.396e-01  +4e-04  6e-07  8e-07  3e-07  7e-06  0.7696  6e-02   2  1  1 |  0  0
12  -2.396e-01  -2.396e-01  +7e-05  1e-07  1e-07  5e-08  1e-06  0.9056  1e-01   2  1  1 |  0  0
13  -2.396e-01  -2.396e-01  +1e-05  3e-08  3e-08  1e-08  3e-07  0.8121  3e-02   2  1  1 |  0  0
14  -2.396e-01  -2.396e-01  +2e-06  3e-09  4e-09  1e-09  4e-08  0.9434  8e-02   2  1  1 |  0  0
15  -2.396e-01  -2.396e-01  +3e-07  5e-10  7e-10  2e-10  6e-09  0.8483  1e-02   3  1  1 |  0  0
16  -2.396e-01  -2.396e-01  +4e-08  6e-11  8e-11  3e-11  8e-10  0.9538  8e-02   2  1  1 |  0  0
17  -2.396e-01  -2.396e-01  +5e-09  9e-12  1e-11  4e-12  1e-10  0.8696  6e-03   2  1  1 |  0  0

OPTIMAL (within feastol=1.1e-11, reltol=2.1e-08, abstol=4.9e-09).
Runtime: 0.007260 seconds.


ECOS 2.0.10 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -1.309e-02  -1.918e-01  +6

 8  -3.340e-01  -3.340e-01  +1e-03  4e-06  8e-07  2e-06  2e-05  0.9(CVXPY) Jul 18 03:42:06 PM: Compilation took 6.218e-02 seconds
(CVXPY) Jul 18 03:42:06 PM: Solver (including time spent in interface) took 1.129e-02 seconds
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jul 18 03:42:06 PM: Your problem has 47 variables, 2 constraints, and 0 parameters.
(CVXPY) Jul 18 03:42:06 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 18 03:42:06 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 18 03:42:06 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
--------------------

(CVXPY) Jul 18 03:42:07 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> ECOS
(CVXPY) Jul 18 03:42:07 PM: Applying reduction FlipObjective
(CVXPY) Jul 18 03:42:07 PM: Applying reduction Dcp2Cone
(CVXPY) Jul 18 03:42:07 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 18 03:42:07 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Jul 18 03:42:07 PM: Applying reduction ECOS
(CVXPY) Jul 18 03:42:07 PM: Finished problem compilation (took 6.831e-02 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Jul 18 03:42:07 PM: Invoking solver ECOS  to obtain a solution.
-------------------------------------------------------------------------------
                                    Summary                                    
-----------------------------

(CVXPY) Jul 18 03:42:07 PM: Optimal value: 1.873e-01
(CVXPY) Jul 18 03:42:07 PM: Compilation took 8.934e-02 seconds
(CVXPY) Jul 18 03:42:07 PM: Solver (including time spent in interface) took 5.401e-03 seconds
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jul 18 03:42:07 PM: Your problem has 47 variables, 2 constraints, and 0 parameters.
(CVXPY) Jul 18 03:42:07 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 18 03:42:07 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 18 03:42:07 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
----------------------------------

In [20]:
weights.to_csv(f'weights.csv')